In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#30 us stocks, you can take any
stocks = ["AXP","AAPL","BA","CAT","CVX","CSCO","DIS","DOW", "XOM",
           "HD","IBM","INTC","JNJ","KO","MCD","MMM","MRK","MSFT",
           "NKE","PFE","PG","TRV","UTX","UNH","VZ","V","WMT","WBA"]

In [3]:
# Dictionary to hold information for each stock/ticker
financial_dir = {}

for ticker in stocks:
    try:
    #getting balance sheet data from yahoo finance for the given ticker
        temp_dir = {}
        url = 'https://in.finance.yahoo.com/quote/'+ticker+'/balance-sheet?p='+ticker
        page = requests.get(url)
        page_content = page.content
        soup = BeautifulSoup(page_content,'html.parser')
        tabl = soup.find_all("div", {"class" : "M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"})
        for t in tabl:
            rows = t.find_all("div", {"class" : "rw-expnded"})
            for row in rows:
                temp_dir[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[1]
        
        #getting income statement data from yahoo finance for the given ticker
        url = 'https://in.finance.yahoo.com/quote/'+ticker+'/financials?p='+ticker
        page = requests.get(url)
        page_content = page.content
        soup = BeautifulSoup(page_content,'html.parser')
        tabl = soup.find_all("div", {"class" : "M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"})
        for t in tabl:
            rows = t.find_all("div", {"class" : "rw-expnded"})
            for row in rows:
                temp_dir[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[1]
        
        #getting cashflow statement data from yahoo finance for the given ticker
        url = 'https://in.finance.yahoo.com/quote/'+ticker+'/cash-flow?p='+ticker
        page = requests.get(url)
        page_content = page.content
        soup = BeautifulSoup(page_content,'html.parser')
        tabl = soup.find_all("div", {"class" : "M(0) Whs(n) BdEnd Bdc($seperatorColor) D(itb)"})
        for t in tabl:
            rows = t.find_all("div", {"class" : "rw-expnded"})
            for row in rows:
                temp_dir[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[1]
        
        #getting key statistics data from yahoo finance for the given ticker
        url = 'https://in.finance.yahoo.com/quote/'+ticker+'/key-statistics?p='+ticker
        page = requests.get(url)
        page_content = page.content
        soup = BeautifulSoup(page_content,'html.parser')
        tabl = soup.findAll("table", {"class": "W(100%) Bdcl(c)"})  
        for t in tabl:
            rows = t.find_all("tr")
            for row in rows:
                if len(row.get_text(separator='|').split("|")[0:2])>0:
                    temp_dir[row.get_text(separator='|').split("|")[0]]=row.get_text(separator='|').split("|")[-1]    
        

        #combining all extracted information with the corresponding ticker into the dictionary
        financial_dir[ticker] = temp_dir
        print("Data successfully scraped for ", ticker)
    except:
        print("Problem scraping data for ",ticker)
print('All data webscraped')

Data successfully scraped for  AXP
Data successfully scraped for  AAPL
Data successfully scraped for  BA
Data successfully scraped for  CAT
Data successfully scraped for  CVX
Data successfully scraped for  CSCO
Data successfully scraped for  DIS
Data successfully scraped for  DOW
Data successfully scraped for  XOM
Data successfully scraped for  HD
Data successfully scraped for  IBM
Data successfully scraped for  INTC
Data successfully scraped for  JNJ
Data successfully scraped for  KO
Data successfully scraped for  MCD
Data successfully scraped for  MMM
Data successfully scraped for  MRK
Data successfully scraped for  MSFT
Data successfully scraped for  NKE
Data successfully scraped for  PFE
Data successfully scraped for  PG
Data successfully scraped for  TRV
Data successfully scraped for  UTX
Data successfully scraped for  UNH
Data successfully scraped for  VZ
Data successfully scraped for  V
Data successfully scraped for  WMT
Data successfully scraped for  WBA
All data webscraped


In [4]:
# Storing the scraped info to a Dataframe
combined_financials = pd.DataFrame(financial_dir)
combined_financials.dropna(how='all',axis=1,inplace=True) #dropping columns with all Null values
tickers = combined_financials.columns #updating the tickers list based on only those tickers whose values were successfully extracted
for ticker in stocks:
    combined_financials = combined_financials[~combined_financials[ticker].str.contains("[a-z]").fillna(False)]

In [5]:
combined_financials.head()

AXP          AAPL  \
Total assets                            19,83,21,000  33,85,16,000   
Total liabilities                       17,52,50,000  24,80,28,000   
Common stock                                1,63,000   4,51,74,000   
Retained earnings                        1,38,71,000   4,58,98,000   
Accumulated other comprehensive income    -27,37,000     -5,84,000   

                                                  BA          CAT  \
Total assets                            13,36,25,000  7,84,53,000   
Total liabilities                       14,19,25,000  6,38,24,000   
Common stock                               50,61,000    59,35,000   
Retained earnings                        5,06,44,000  3,44,37,000   
Accumulated other comprehensive income  -1,61,53,000   -15,67,000   

                                                 CVX         CSCO  \
Total assets                            23,74,28,000  9,48,53,000   
Total liabilities                        9,22,20,000  5,69,33,000   
Common stock                               18,32,000            -   
Retained earnings                       17,49,45,000            -   
Accumulated other comprehensive income    -52,30,000            -   

                                                 DIS           DOW  \
Total assets                            19,39,84,000   6,05,24,000   
Total liabilities                       10,00,95,000   4,64,30,000   
Common stock                             5,39,07,000         8,000   
Retained earnings                        4,24,94,000   1,70,45,000   
Accumulated other comprehensive income    -66,17,000  -1,02,46,000   

                                                 XOM           HD  ...  \
Total assets                            36,25,97,000  5,12,36,000  ...   
Total liabilities                       16,36,59,000  5,43,52,000  ...   
Common stock                             1,56,37,000       89,000  ...   
Retained earnings                       42,13,41,000  5,17,29,000  ...   
Accumulated other comprehensive income  -1,94,93,000    -7,39,000  ...   

                                                NKE           PFE  \
Total assets                            3,13,42,000  16,74,89,000   
Total liabilities                       2,32,87,000  10,40,42,000   
Common stock                                  3,000      4,85,000   
Retained earnings                         -1,91,000   9,76,70,000   
Accumulated other comprehensive income      -56,000  -1,16,40,000   

                                                  PG           TRV  \
Total assets                            12,07,00,000  11,01,22,000   
Total liabilities                        7,38,22,000   8,41,79,000   
Common stock                               49,06,000   2,34,69,000   
Retained earnings                       10,02,39,000   3,69,77,000   
Accumulated other comprehensive income  -1,72,45,000      6,40,000   

                                                 UTX           UNH  \
Total assets                            13,97,16,000  17,38,89,000   
Total liabilities                        9,54,85,000  11,34,53,000   
Common stock                             2,30,19,000         9,000   
Retained earnings                        6,15,94,000   6,11,78,000   
Accumulated other comprehensive income  -1,01,49,000    -35,78,000   

                                                  VZ            V  \
Total assets                            29,17,27,000  7,25,74,000   
Total liabilities                       22,88,92,000  3,78,90,000   
Common stock                                4,29,000    54,62,000   
Retained earnings                        5,31,47,000  1,35,02,000   
Accumulated other comprehensive income      9,98,000    -6,50,000   

                                                 WMT          WBA  
Total assets                            23,64,95,000  6,75,98,000  
Total liabilities                       15,49,43,000  4,34,46,000  
Common stock                                2,84,000    

In [6]:
# Required stats for the magic formula
stats = ["EBITDA",
         "Depreciation & amortisation",
         "Market cap (intra-day)",
         "Net income available to common shareholders",
         "Net cash provided by operating activities",
         "Capital expenditure",
         "Total current assets",
         "Total current liabilities",
         "Net property, plant and equipment",
         "Total stockholders' equity",
         "Long-term debt",
         "Forward annual dividend yield"]

# Short hands for above to keep things simple
indx = ["EBITDA","D&A","MarketCap","NetIncome","CashFlowOps","Capex","CurrAsset",
        "CurrLiab","PPE","BookValue","TotDebt","DivYield"]

In [7]:
all_stats = {}
for ticker in stocks:
    try:
        temp = combined_financials[ticker]
        ticker_stats = []
        for stat in stats:
            ticker_stats.append(temp.loc[stat])
        all_stats['{}'.format(ticker)] = ticker_stats
    except:
        print("Can't read data for ",ticker)

### Combining everything into one single dataframe with all required stats and some data cleaning


In [8]:
all_stats_df = pd.DataFrame(all_stats,index=indx)
all_stats_df[tickers] = all_stats_df[tickers].replace({',': ''}, regex=True)
all_stats_df[tickers] = all_stats_df[tickers].replace({'M': 'E+03'}, regex=True)
all_stats_df[tickers] = all_stats_df[tickers].replace({'B': 'E+06'}, regex=True)
all_stats_df[tickers] = all_stats_df[tickers].replace({'T': 'E+09'}, regex=True)
all_stats_df[tickers] = all_stats_df[tickers].replace({'%': 'E-02'}, regex=True)
for ticker in all_stats_df.columns:
    all_stats_df[ticker] = pd.to_numeric(all_stats_df[ticker].values,errors='coerce')
all_stats_df.dropna(axis=1,inplace=True)

stocks = all_stats_df.columns

In [11]:
all_stats_df

AAPL           CAT          CSCO           DOW  \
EBITDA       7.867000e+07  8.580000e+06  1.591000e+07  5.680000e+06   
D&A          1.153300e+07  2.511000e+06  1.808000e+06  2.876000e+06   
MarketCap    2.160000e+09  7.680000e+07  1.783900e+08  3.434000e+07   
NetIncome    5.842400e+07  4.142000e+06  1.121400e+07 -1.976000e+06   
CashFlowOps  8.000800e+07  5.724000e+06  1.542600e+07  6.503000e+06   
Capex       -8.302000e+06 -2.442000e+06 -7.700000e+05 -1.717000e+06   
CurrAsset    1.628190e+08  3.919300e+07  4.357300e+07  1.681500e+07   
CurrLiab     1.057180e+08  2.662100e+07  2.533100e+07  1.067900e+07   
PPE          3.737800e+07  1.290400e+07  2.453000e+06  2.306800e+07   
BookValue    9.048800e+07  1.458800e+07  3.792000e+07  1.354100e+07   
TotDebt      9.180700e+07  2.615300e+07  1.157800e+07  1.597500e+07   
DivYield     6.600000e-03  2.920000e-02  3.430000e-02  6.070000e-02   

                       HD           IBM          INTC           JNJ  \
EBITDA       1.910000e+07  1.558000e+07  3.812000e+07  2.721000e+07   
D&A          2.411000e+06  6.632000e+06  1.176000e+07  7.016000e+06   
MarketCap    3.139800e+08  1.105800e+08  2.107400e+08  4.009800e+08   
NetIncome    1.182700e+07  7.878000e+06  2.366100e+07  1.518500e+07   
CashFlowOps  2.005500e+07  1.512200e+07  3.791400e+07  2.073500e+07   
Capex       -2.464000e+06 -2.996000e+06 -1.601400e+07 -3.292000e+06   
CurrAsset    1.981000e+07  3.842000e+07  3.123900e+07  4.527400e+07   
CurrLiab     1.837500e+07  3.770100e+07  2.231000e+07  3.596400e+07   
PPE          2.836500e+07  1.500500e+07  5.538600e+07  1.765800e+07   
BookValue   -3.116000e+06  2.084100e+07  7.765900e+07  5.947100e+07   
TotDebt      2.867000e+07  5.410200e+07  2.530800e+07  2.649400e+07   
DivYield     2.100000e-02  5.230000e-02  2.670000e-02  2.660000e-02   

                       KO           MCD  ...           MRK          MSFT  \
EBITDA       1.125000e+07  8.910000e+06  ...  1.905000e+07  6.526000e+07   
D&A          1.511000e+06  1.674000e+06  ...  3.650000e+06  1.279600e+07   
MarketCap    2.068700e+08  1.590600e+08  ...  2.163500e+08  1.670000e+09   
NetIncome    9.189000e+06  4.770800e+06  ...  1.047900e+07  4.428100e+07   
CashFlowOps  8.756000e+06  5.508900e+06  ...  1.309400e+07  6.067500e+07   
Capex       -1.823000e+06 -2.068100e+06  ... -3.750000e+06 -1.544100e+07   
CurrAsset    2.041100e+07  3.557900e+06  ...  2.748300e+07  1.819150e+08   
CurrLiab     2.697300e+07  3.621000e+06  ...  2.222000e+07  7.231000e+07   
PPE          1.083800e+07  3.742120e+07  ...  1.505300e+07  5.290400e+07   
BookValue    1.898100e+07 -8.210300e+06  ...  2.590700e+07  1.183040e+08   
TotDebt      2.751600e+07  3.411810e+07  ...  2.273600e+07  5.957800e+07   
DivYield     3.420000e-02  2.350000e-02  ...  2.850000e-02  9.400000e-03   

                      NKE           PFE            PG           UNH  \
EBITDA       4.230000e+06  2.005000e+07  1.919000e+07  2.649000e+07   
D&A          7.140000e+05  5.406000e+06  3.013000e+06  2.867000e+06   
MarketCap    1.739700e+08  2.114400e+08  3.440500e+08  2.934800e+08   
NetIncome    4.304000e+06  1.417100e+07  1.276400e+07  1.709800e+07   
CashFlowOps  4.996000e+06  1.496700e+07  1.740300e+07  2.230100e+07   
Capex       -1.086000e+06 -2.363000e+06 -3.073000e+06 -2.014000e+06   
CurrAsset    2.055600e+07  3.280300e+07  2.798700e+07  4.263400e+07   
CurrLiab     8.284000e+06  3.730400e+07  3.297600e+07  6.178200e+07   
PPE          7.963000e+06  1.396700e+07  2.069200e+07  8.704000e+06   
BookValue    8.055000e+06  6.314300e+07  4.652100e+07  5.761600e+07   
TotDebt      9.406000e+06  3.595500e+07  2.353700e+07  3.680800e+07   
DivYield     8.800000e-03  3.960000e-02  2.270000e-02  1.600000e-02   

                       VZ             V           WMT           WBA  
EBITDA       4.751000e+07  1.548000e+07  3.382000e+07  5.020000e+06  
D&A          1.655000e+07  7.430000e+05  1.111300e+07  1.973000e+06  
MarketCap    2.460500e+08  4.616100e+08  3.701400e

## Calculation of some metrics for the formula using above information

In [12]:
transpose_df = all_stats_df.transpose()
final_stats_df = pd.DataFrame()
final_stats_df["EBIT"] = transpose_df["EBITDA"] - transpose_df["D&A"]
final_stats_df["TEV"] =  transpose_df["MarketCap"].fillna(0) \
                         +transpose_df["TotDebt"].fillna(0) \
                         -(transpose_df["CurrAsset"].fillna(0)-transpose_df["CurrLiab"].fillna(0))
final_stats_df["EarningYield"] =  final_stats_df["EBIT"]/final_stats_df["TEV"]
final_stats_df["FCFYield"] = (transpose_df["CashFlowOps"]-transpose_df["Capex"])/transpose_df["MarketCap"]
final_stats_df["ROC"]  = (transpose_df["EBITDA"] - transpose_df["D&A"])/(transpose_df["PPE"]+transpose_df["CurrAsset"]-transpose_df["CurrLiab"])
final_stats_df["BookToMkt"] = transpose_df["BookValue"]/transpose_df["MarketCap"]
final_stats_df["DivYield"] = transpose_df["DivYield"]

In [13]:
final_stats_df

EBIT           TEV  EarningYield  FCFYield       ROC  BookToMkt  \
AAPL  67137000.0  2.194706e+09      0.030590  0.040884  0.710602   0.041893   
CAT    6069000.0  9.038100e+07      0.067149  0.106328  0.238224   0.189948   
CSCO  14102000.0  1.717260e+08      0.082119  0.090790  0.681421   0.212568   
DOW    2804000.0  4.417900e+07      0.063469  0.239371  0.096014   0.394321   
HD    16689000.0  3.412150e+08      0.048911  0.071721  0.560034  -0.009924   
IBM    8948000.0  1.639630e+08      0.054573  0.163845  0.569066   0.188470   
INTC  26360000.0  2.271190e+08      0.116063  0.255898  0.409858   0.368506   
JNJ   20194000.0  4.181640e+08      0.048292  0.059921  0.748813   0.148314   
KO     9739000.0  2.409480e+08      0.040420  0.051138  2.277596   0.091753   
MCD    7236000.0  1.932412e+08      0.037445  0.047636  0.193693  -0.051618   
MMM    6676000.0  1.081790e+08      0.061713  0.096134  0.478910   0.106588   
MRK   15400000.0  2.338230e+08      0.065862  0.077855  0.758023   0.119746   
MSFT  52464000.0  1.619973e+09      0.032386  0.045578  0.322838   0.070841   
NKE    3516000.0  1.711040e+08      0.020549  0.034960  0.173758   0.046301   
PFE   14644000.0  2.518960e+08      0.058135  0.081962  1.547010   0.298633   
PG    16177000.0  3.725760e+08      0.043419  0.059515  1.030185   0.135216   
UNH   23623000.0  3.494360e+08      0.067603  0.082851 -2.261873   0.196320   
VZ    30960000.0  3.533770e+08      0.087612  0.267360  0.288768   0.249522   
V     14737000.0  4.707840e+08      0.031303  0.028602  1.437756   0.075137   
WMT   22707000.0  4.298380e+08      0.052827  0.114630  0.204448   0.201732   
WBA    3047000.0  5.167700e+07      0.058962  0.214712  0.475351   0.701641   

      DivYield  
AAPL    0.0066  
CAT     0.0292  
CSCO    0.0343  
DOW     0.0607  
HD      0.0210  
IBM     0.0523  
INTC    0.0267  
JNJ     0.0266  
KO      0.0342  
MCD     0.0235  
MMM     0.0361  
MRK     0.0285  
MSFT    0.0094  
NKE     0.0088  
PFE     0.0396  
PG      0.0227  
UNH     0.0160  
VZ      0.0414  
V       0.0058  
WMT     0.0165  
WBA     0.0473

#### We now have everything in place to apply the magic formula, let's do it!

### Ranking using earning yield and ROC(Return on Capital)

In [15]:
final_stats_val_df = final_stats_df.loc[stocks,:]
final_stats_val_df["CombRank"] = final_stats_val_df["EarningYield"].rank(ascending=False,na_option='bottom')+final_stats_val_df["ROC"].rank(ascending=False,na_option='bottom')
final_stats_val_df["MagicFormulaRank"] = final_stats_val_df["CombRank"].rank(method='first')
value_stocks = final_stats_val_df.sort_values("MagicFormulaRank").iloc[:,[2,4,8]]
print("------------------------------------------------")
print("Value stocks based on Greenblatt's Magic Formula")
df = value_stocks.fillna(0)
df

------------------------------------------------
Value stocks based on Greenblatt's Magic Formula


EarningYield       ROC  MagicFormulaRank
CSCO      0.082119  0.681421               1.0
MRK       0.065862  0.758023               2.0
PFE       0.058135  1.547010               3.0
INTC      0.116063  0.409858               4.0
KO        0.040420  2.277596               5.0
VZ        0.087612  0.288768               6.0
MMM       0.061713  0.478910               7.0
PG        0.043419  1.030185               8.0
IBM       0.054573  0.569066               9.0
JNJ       0.048292  0.748813              10.0
CAT       0.067149  0.238224              11.0
WBA       0.058962  0.475351              12.0
V         0.031303  1.437756              13.0
HD        0.048911  0.560034              14.0
UNH       0.067603 -2.261873              15.0
AAPL      0.030590  0.710602              16.0
DOW       0.063469  0.096014              17.0
WMT       0.052827  0.204448              18.0
MSFT      0.032386  0.322838              19.0
MCD       0.037445  0.193693              20.0
NKE       0.020549  0.173758              21.0

Hmm, looks like Cisco, Merk Pharma and Pfizer are the top 3 picks, based on earning yield and ROC

### Ranking using dividend yield(not included in Magic formula)

Although I personally invest in stocks for value, not for dividends, they are an extra incentive sometimes! :)

In [17]:
high_dividend_stocks = final_stats_df.sort_values("DivYield",ascending=False).iloc[:,6]
print("------------------------------------------------")
print("Highest dividend paying stocks")
high_dividend_stocks.to_frame()

------------------------------------------------
Highest dividend paying stocks


DivYield
DOW     0.0607
IBM     0.0523
WBA     0.0473
VZ      0.0414
PFE     0.0396
MMM     0.0361
CSCO    0.0343
KO      0.0342
CAT     0.0292
MRK     0.0285
INTC    0.0267
JNJ     0.0266
MCD     0.0235
PG      0.0227
HD      0.0210
WMT     0.0165
UNH     0.0160
MSFT    0.0094
NKE     0.0088
AAPL    0.0066
V       0.0058

## Conclusion

Combining both the magic formula and dividend yield.


In [18]:
final_stats_df["CombRank"] = final_stats_df["EarningYield"].rank(ascending=False,method='first') \
                              +final_stats_df["ROC"].rank(ascending=False,method='first')  \
                              +final_stats_df["DivYield"].rank(ascending=False,method='first')
final_stats_df["CombinedRank"] = final_stats_df["CombRank"].rank(method='first')
value_high_div_stocks = final_stats_df.sort_values("CombinedRank").iloc[:,[2,4,6,8]]
print("------------------------------------------------")
print("Magic Formula and Dividend Yield combined")
value_high_div_stocks

------------------------------------------------
Magic Formula and Dividend Yield combined


EarningYield       ROC  DivYield  CombinedRank
PFE       0.058135  1.547010    0.0396           1.0
CSCO      0.082119  0.681421    0.0343           2.0
MRK       0.065862  0.758023    0.0285           3.0
VZ        0.087612  0.288768    0.0414           4.0
IBM       0.054573  0.569066    0.0523           5.0
WBA       0.058962  0.475351    0.0473           6.0
INTC      0.116063  0.409858    0.0267           7.0
KO        0.040420  2.277596    0.0342           8.0
MMM       0.061713  0.478910    0.0361           9.0
DOW       0.063469  0.096014    0.0607          10.0
CAT       0.067149  0.238224    0.0292          11.0
JNJ       0.048292  0.748813    0.0266          12.0
PG        0.043419  1.030185    0.0227          13.0
HD        0.048911  0.560034    0.0210          14.0
UNH       0.067603 -2.261873    0.0160          15.0
V         0.031303  1.437756    0.0058          16.0
WMT       0.052827  0.204448    0.0165          17.0
AAPL      0.030590  0.710602    0.0066          18.0
MCD       0.037445  0.193693    0.0235          19.0
MSFT      0.032386  0.322838    0.0094          20.0
NKE       0.020549  0.173758    0.0088          21.0

#### DISCLAIMER : Please do not use these results to make actual investments, I did this just out of curoisity, do your own research and make decisions based on that, happy investing! :)